In [2]:
import rdflib
from rdflib.plugins import sparql
import re
import numpy as np 

In [3]:
# query_text = """
# PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
# PREFIX schema: <http://schema.org/>
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# SELECT ?person ?name WHERE {
#   ?person rdf:type schema:Person ;
#   		schema:height ?height;
#     	schema:givenName ?name .
#   filter(?height>170)
# } 
# ORDER BY ?name ?height
# """
# query  = sparql.processor.prepareQuery(query_text)

In [4]:
query_text = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX lab: <https://mini.pw.edu.pl/sdp/lab3/>
SELECT ?firstPerson (?secondPerson as ?thirdPerson) (AVG(?food) as ?anyFood) (GROUP_CONCAT(?food) as ?minfood )  {
  	?firstPerson rdf:type schema:Person ;
  				skos:subject ?food .
	?secondPerson  rdf:type schema:Person ; 
                skos:subject ?food .
  filter(?firstPerson != ?secondPerson && STR(?firstPerson) < STR(?secondPerson))
}
group by ?firstPerson ?secondPerson
ORDER BY ?firstPerson 
"""

query  = sparql.processor.prepareQuery(query_text)

In [5]:
parse_results = sparql.parser.parseQuery(query_text)
algebra_result = sparql.algebra.translateQuery(parse_results)
sparql.algebra.pprintAlgebra(algebra_result)

SelectQuery(
    p = Project(
        p = OrderBy(
            p = Extend(
                p = Extend(
                    p = Extend(
                        p = Extend(
                            p = AggregateJoin(
                                A = [Aggregate_Sample_{'vars': rdflib.term.Variable('secondPerson'), 'res': rdflib.term.Variable('__agg_1__'), '_vars': {rdflib.term.Variable('__agg_1__'), rdflib.term.Variable('secondPerson')}}, Aggregate_Avg_{'distinct': [], 'vars': rdflib.term.Variable('food'), 'res': rdflib.term.Variable('__agg_2__'), '_vars': {rdflib.term.Variable('__agg_2__'), rdflib.term.Variable('food')}}, Aggregate_GroupConcat_{'distinct': [], 'vars': rdflib.term.Variable('food'), 'res': rdflib.term.Variable('__agg_3__'), '_vars': {rdflib.term.Variable('food'), rdflib.term.Variable('__agg_3__')}}, Aggregate_Sample_{'vars': rdflib.term.Variable('firstPerson'), 'res': rdflib.term.Variable('__agg_4__'), '_vars': {rdflib.term.Variable('__agg_4__'), rdflib.term.Variable

In [7]:
algebra_dict = dict(algebra_result.algebra)

In [19]:
algebra_dict['p']['p']['p']['p'].keys()

odict_keys(['p', 'expr', 'var', '_vars'])

In [32]:
algebra_dict['p']['p']['p']['p']['p']['p']['p']['A']

[Aggregate_Sample_{'vars': rdflib.term.Variable('secondPerson'), 'res': rdflib.term.Variable('__agg_1__'), '_vars': {rdflib.term.Variable('__agg_1__'), rdflib.term.Variable('secondPerson')}},
 Aggregate_Avg_{'distinct': [], 'vars': rdflib.term.Variable('food'), 'res': rdflib.term.Variable('__agg_2__'), '_vars': {rdflib.term.Variable('__agg_2__'), rdflib.term.Variable('food')}},
 Aggregate_GroupConcat_{'distinct': [], 'vars': rdflib.term.Variable('food'), 'res': rdflib.term.Variable('__agg_3__'), '_vars': {rdflib.term.Variable('food'), rdflib.term.Variable('__agg_3__')}},
 Aggregate_Sample_{'vars': rdflib.term.Variable('firstPerson'), 'res': rdflib.term.Variable('__agg_4__'), '_vars': {rdflib.term.Variable('__agg_4__'), rdflib.term.Variable('firstPerson')}}]

In [8]:
def get_literal(literal):
    type = literal.datatype
    if type is not None:
        return {'value':literal.toPython(),"type":literal.datatype.toPython()}
    else:
        return {'value':literal.toPython(),"type":literal.datatype}

In [9]:
def get_variable(variable):
    return variable.toPython()

In [10]:
def parse_filterings(expression):
    if expression.name == 'RelationalExpression':
        if isinstance(expression['other'], rdflib.term.Literal):
            other = get_literal(expression['other'])
        else:
            other = get_variable(expression['other'])
        if isinstance(expression['expr'], rdflib.term.Literal):
            expr = get_literal(expression['expr'])
        else:
            expr = get_variable(expression['expr'])
        return {
            'op': expression['op'],
            'expr': expr,
            'other': other,
        }
    elif expression.name == 'ConditionalAndExpression':
        parsed_expression = {
            'logic' : 'and',
            'expr': parse_filterings(expression['expr']),
            'other': parse_filterings(expression['other'][0])
        }
        return parsed_expression
    elif expression.name == 'ConditionalOrExpression':
        parsed_expression = {
            'logic' : 'or',
            'expr': parse_filterings(expression['expr']),
            'other': parse_filterings(expression['other'][0])
        }
        return parsed_expression
    else:
        return None 

In [55]:
def transform_aggregates(s):
    return s.replace('?__agg_', '?agg').replace('__', '')

In [11]:
def parse_triple(triple):
    return {
        'subject': triple[0].toPython(),
        'predicate': triple[1].toPython(),
        'object': triple[2].toPython()
    }

In [64]:
def parse_extends(expression):
    extends = []
    while expression.name == 'Extend':
        extends.append({'expr': transform_aggregates(expression['expr'].toPython()),'var':expression['var'].toPython()})
        expression = expression['p']
    return extends

In [56]:
# def parse_groupby(expression):
#     parsed_expression = {
#         'variables': [],
#         'aggregates': []
#     }
#     aggregated_variables = []
#     parsed_expression['variables'] = [variable.toPython() for variable in expression['p']['expr']]
    
#     aggregate_mapping = {
#         'Aggregate_Sample': 'Sample',
#         'Aggregate_Min': 'Min',
#         'Aggregate_Max': 'Max',
#         'Aggregate_Count': 'Count',
#         'Aggregate_Sum': 'Sum',
#         'Aggregate_Avg': 'Avg',
#         'Aggregate_GroupConcat': 'GroupConcat'
#     }
    
#     for agg in expression['A']:
#         agg_name = agg.name
#         if agg_name in aggregate_mapping:
#             parsed_expression['aggregates'].append({
#                 'type': aggregate_mapping[agg_name],
#                 'variable': agg['vars'].toPython(),
#                 'res': transform_aggregates(agg['res'].toPython())
#             })
#             aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
    
#     return parsed_expression, aggregated_variables

In [60]:
def parse_groupby(expression):
    
    parsed_expression = {
        'variables' :[],
        'aggregates': []
    } 
    aggregated_variables = []
    parsed_expression['variables'] = [variable.toPython() for variable in expression['p']['expr']]
    for agg in expression['A']:
        if agg.name == 'Aggregate_Sample' and 'distinct' in agg.keys():
            parsed_expression['aggregates'].append( {
                'type':'Sample',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(transform_aggregates(agg['res'].toPython()))
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
        elif agg.name == 'Aggregate_Min':
            parsed_expression['aggregates'].append( {
                'type':'Min',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(transform_aggregates(agg['res'].toPython()))
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
        elif agg.name == 'Aggregate_Max':
            parsed_expression['aggregates'].append( {
                'type':'Max',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(agg['res'].toPython())
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
        elif agg.name == 'Aggregate_Count':
            parsed_expression['aggregates'].append( {
                'type':'Count',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(agg['res'].toPython())
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
        elif agg.name == 'Aggregate_Sum':
            parsed_expression['aggregates'].append( {
                'type':'Sum',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(agg['res'].toPython())
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
        elif agg.name == 'Aggregate_Avg':
            parsed_expression['aggregates'].append( {
                'type':'Avg',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(agg['res'].toPython())
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
        elif agg.name == 'Aggregate_GroupConcat':
            parsed_expression['aggregates'].append( {
                'type':'GroupConcat',
                'variable': agg['vars'].toPython(),
                'res' : transform_aggregates(agg['res'].toPython())
            })
            aggregated_variables.append(transform_aggregates(agg['res'].toPython()))
    return parsed_expression,aggregated_variables
        

In [62]:
def convert_to_query_structure(input_dict):
    query_dict = {
    }
    tmp_dict = input_dict['p']
    project = tmp_dict['PV']
    query_dict['project'] = [p.toPython() for p in project]
    
    try:
        if tmp_dict['p'].name=='OrderBy':
            order_condition = tmp_dict['p']['expr']
            query_dict['order'] = []
            for condition in order_condition:
                query_dict['order'].append(condition['expr'].toPython())
            tmp_dict = tmp_dict['p']
    except KeyError:
        pass
    try:
        if tmp_dict['p'].name=='Extend':
            extends = parse_extends(tmp_dict['p'])
            for i in range(len(query_dict['project'])+1):
                tmp_dict = tmp_dict['p']
            grouped,aggregates = parse_groupby(tmp_dict)
            print(aggregates)
            query_dict['extend'] =  [item for item in extends if item['expr'] in aggregates]
            query_dict['group'] = grouped
            tmp_dict = tmp_dict['p']['p']
    except KeyError:
        pass
    
    try:
        filter_condition = tmp_dict['p']['expr']
        query_dict['filter'] = parse_filterings(filter_condition)
        tmp_dict = tmp_dict['p']
    except KeyError:
        pass
    
    try:
        bgp_triples = tmp_dict['p']['triples']
        query_dict['bgp'] = []
        for triple in bgp_triples:
            query_dict['bgp'].append(parse_triple(triple))
    except KeyError:
        pass

    return query_dict


In [65]:
convert_to_query_structure(algebra_dict)

['?agg2', '?agg3']


{'project': ['?firstPerson', '?thirdPerson', '?anyFood', '?minfood'],
 'order': ['?firstPerson'],
 'extend': [{'expr': '?agg3', 'var': '?minfood'},
  {'expr': '?agg2', 'var': '?anyFood'}],
 'group': {'variables': ['?firstPerson', '?secondPerson'],
  'aggregates': [{'type': 'Avg', 'variable': '?food', 'res': '?agg2'},
   {'type': 'GroupConcat', 'variable': '?food', 'res': '?agg3'}]},
 'bgp': [{'subject': '?firstPerson',
   'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
   'object': 'http://schema.org/Person'},
  {'subject': '?secondPerson',
   'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
   'object': 'http://schema.org/Person'},
  {'subject': '?firstPerson',
   'predicate': 'http://www.w3.org/2004/02/skos/core#subject',
   'object': '?food'},
  {'subject': '?secondPerson',
   'predicate': 'http://www.w3.org/2004/02/skos/core#subject',
   'object': '?food'}]}

In [15]:
def parse_prefixes(prefix_string):
    prefixes = {}
    pattern = re.compile(r'PREFIX\s+(\w+):\s*<(.+?)>')
    matches = pattern.findall(prefix_string)
    for match in matches:
        prefix, uri = match
        prefixes[prefix] = uri
    return prefixes

In [16]:
def convert_query_text(query_text):
    query_dict = {
        'prefixes': {},
        'query': {}
    }
    parse_results = sparql.parser.parseQuery(query_text)
    algebra_result = sparql.algebra.translateQuery(parse_results)
    algebra_dict = dict(algebra_result.algebra)
    query_dict['prefixes'] = parse_prefixes(query_text)
    query_dict['query'] = convert_to_query_structure(algebra_dict)
    return query_dict   

In [17]:
convert_query_text(query_text)

AggregateJoin_AggregateJoin_{'A': [Aggregate_Sample_{'vars': rdflib.term.Variable('secondPerson'), 'res': rdflib.term.Variable('__agg_1__'), '_vars': {rdflib.term.Variable('__agg_1__'), rdflib.term.Variable('secondPerson')}}, Aggregate_Avg_{'distinct': [], 'vars': rdflib.term.Variable('food'), 'res': rdflib.term.Variable('__agg_2__'), '_vars': {rdflib.term.Variable('__agg_2__'), rdflib.term.Variable('food')}}, Aggregate_GroupConcat_{'distinct': [], 'vars': rdflib.term.Variable('food'), 'res': rdflib.term.Variable('__agg_3__'), '_vars': {rdflib.term.Variable('food'), rdflib.term.Variable('__agg_3__')}}, Aggregate_Sample_{'vars': rdflib.term.Variable('firstPerson'), 'res': rdflib.term.Variable('__agg_4__'), '_vars': {rdflib.term.Variable('__agg_4__'), rdflib.term.Variable('firstPerson')}}], 'p': Group_{'p': Filter_{'expr': ConditionalAndExpression_{'expr': RelationalExpression_{'expr': rdflib.term.Variable('firstPerson'), 'op': '!=', 'other': rdflib.term.Variable('secondPerson'), '_vars'

{'prefixes': {'xsd': 'http://www.w3.org/2001/XMLSchema#',
  'schema': 'http://schema.org/',
  'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'rdfs': 'http://www.w3.org/2000/01/rdf-schema#',
  'skos': 'http://www.w3.org/2004/02/skos/core#',
  'lab': 'https://mini.pw.edu.pl/sdp/lab3/'},
 'query': {'project': ['?firstPerson', '?thirdPerson', '?anyFood', '?minfood'],
  'order': ['?firstPerson'],
  'group': {'variables': ['?firstPerson', '?secondPerson'],
   'aggregates': [{'type': 'Avg', 'variable': '?food'},
    {'type': 'GroupConcat', 'variable': '?food'}]},
  'bgp': [{'subject': '?firstPerson',
    'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
    'object': 'http://schema.org/Person'},
   {'subject': '?secondPerson',
    'predicate': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
    'object': 'http://schema.org/Person'},
   {'subject': '?firstPerson',
    'predicate': 'http://www.w3.org/2004/02/skos/core#subject',
    'object': '?food'},
   {'subject': '?